#### Compute Similarity
The goal of this notebook is to compute song similarity between a sample of all songs and create necessary files to create a graph in neo4j.

In [9]:
# import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations


In [8]:
# import song data
song_df = pd.read_csv('spotify.csv')
song_df.drop('Unnamed: 0', inplace=True, axis=1)
song_df.head()


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [10]:
#TODO should we normalize?
def normalize(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """
    Normalize specified columns in the DataFrame using Min-Max scaling.

    Args:
        df (DataFrame): DataFrame containing the data to be normalized
        columns (list): List of column names to be normalized

    Returns:
        normalized_df (DataFrame): DataFrame with specified columns normalized
    """
    normalized_df = df.copy()
    for column in columns:
        # Compute min and max values for the column
        min_val = df[column].min()
        max_val = df[column].max()
        # Perform Min-Max scaling
        normalized_df[column] = (df[column] - min_val) / (max_val - min_val)
    return normalized_df

columns_to_normalize = ['popularity', 'energy', 'loudness', 'acousticness',
                        'instrumentalness', 'liveness', 'valence', 'tempo']
norm_df = normalize(song_df, columns_to_normalize)
norm_df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,0.73,230666,False,0.676,0.4610,1,0.791392,0,0.1430,0.032329,0.000001,0.3580,0.718593,0.361245,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,0.55,149610,False,0.420,0.1660,1,0.597377,1,0.0763,0.927711,0.000006,0.1010,0.268342,0.318397,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,0.57,210826,False,0.438,0.3590,0,0.736123,1,0.0557,0.210843,0.000000,0.1170,0.120603,0.313643,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,0.71,201933,False,0.266,0.0596,0,0.573701,1,0.0363,0.908635,0.000071,0.1320,0.143719,0.746758,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,0.82,198853,False,0.618,0.4430,2,0.737103,1,0.0526,0.470884,0.000000,0.0829,0.167839,0.492863,4,acoustic


In [11]:
def row_similarity(row1: pd.Series, row2: pd.Series) -> float:
    """
        Computes similarity between two songs by computing the cosine similarity on
        columns of interest.

        Args:
            row1 (Series): song 1 to compare similarity
            row2 (Series): song 2 to compare similarity
        
        Returns:
        sim_score (float): similarity between two songs
    """ 
    # compute values that represent if the songs have the same artists, album_name, or track_genre
    same_artists = 1 if row1['artists'] == row2['artists'] else 0
    same_album_name = 1 if row1['album_name'] == row2['album_name'] else 0
    same_track_genre = 1 if row1['track_genre'] == row2['track_genre'] else 0

    # compute cosine similarity between two songs
    sim_score = cosine_similarity([[same_artists, same_album_name, same_track_genre,
            row1['popularity'], row1['energy'], row1['loudness'], row1['acousticness'],
            row1['instrumentalness'], row1['liveness'], row1['valence'], row1['tempo']]],
            [[same_artists, same_album_name, same_track_genre,
            row2['popularity'], row2['energy'], row1['loudness'], row1['acousticness'],
            row2['instrumentalness'], row2['liveness'], row2['valence'], row2['tempo']]])[0][0]

    return sim_score

In [12]:
# def compare_all_songs(df: pd.DataFrame) -> pd.DataFrame:
#     """
#         Computes similarity between every pair of unique songs in the given DataFrame.
# 
#         Args:
#             df (DataFrame): dataframe of songs
#         
#         Returns:
#         sim_df (DataFrame): dataframe of unique song pairs and their similarity score
#     """ 
#     # create unique pairs of song rows
#     unique_pairs = [(i, row1, j, row2) for i, row1 in df.iterrows() for j, row2 in df.iterrows()if i<j]
# 
#     # create a list of similarity scores for each unique pair
#     similarity_scores = [row_similarity(row1, row2) for _, row1, _, row2 in unique_pairs]
# 
#     # get track ids and similarity scores for each unique pair
#     track_ids = [(row1['track_id'], row2['track_id']) for _, row1, _, row2 in unique_pairs]
#     sim_scores = similarity_scores  
# 
#     # create a dataframe of similairty scores for each unique pair
#     sim_df = pd.DataFrame({'track_id1': [t[0] for t in track_ids],
#                         'track_id2': [t[1] for t in track_ids],
#                         'sim_score': sim_scores})
#     return sim_df


#TODO I edited yours by adding combinations from itertools to prevent redundant comparisons (like compars song a to b and b to a)
def compare_all_songs(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes similarity between every pair of unique songs in the given DataFrame.

    Args:
        df (DataFrame): DataFrame of songs

    Returns:
        sim_df (DataFrame): DataFrame of unique song pairs and their similarity score
    """
    # Initialize lists to store track IDs and similarity scores
    track_ids = []
    sim_scores = []

    # Iterate over unique pairs of song indices
    for idx1, idx2 in combinations(df.index, 2):
        row1, row2 = df.loc[idx1], df.loc[idx2]
        
        # Compute similarity between the pair
        similarity_score = row_similarity(row1, row2)

        # Store track IDs and similarity score
        track_ids.append((row1['track_id'], row2['track_id']))
        sim_scores.append(similarity_score)

    # Create DataFrame from similarity data
    sim_df = pd.DataFrame({'track_id1': [t[0] for t in track_ids],
                           'track_id2': [t[1] for t in track_ids],
                           'sim_score': sim_scores})
    
    return sim_df

In [13]:
# take a random sample of all songs with a total of 100 songs
sample_df = song_df.sample(n=100, replace=False)
# get all Strokes songs available 
strokes_df = song_df[song_df['artists']=='The Strokes']
# add all Strokes songs to the sample
sample_df = pd.concat([sample_df, strokes_df])
# drop duplicate rows from the sample
sample_df= sample_df.drop_duplicates().reset_index(drop=True)
sample_df.head(3)

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,6KRyTDrXw3InXjfnOB3iCR,Distinguisher,Under the Weight of Things I Couldn't Change,Without End,22,154000,True,0.307,0.995,1,-1.696,1,0.2740,0.00015,0.0416,0.1870,0.114,197.087,4,death-metal
1,5Q23x9RyABJ9vx09lNb9hs,Belmonte & Amaraí,Luar do Sertão - Belmonte & Amaraí,Saudade Da Minha Terra,46,188826,False,0.749,0.414,1,-7.856,1,0.0665,0.70900,0.0000,0.2820,0.851,109.263,3,sertanejo
2,4zqlCicLICAUkPYYqnPbkn,Sajanka;TERRA,Galiya,Galiya,41,386000,False,0.535,0.732,6,-6.269,0,0.0455,0.00584,0.5020,0.0604,0.493,150.025,4,trance


In [14]:
# get similarity scores for each row in the sample
sim_df = compare_all_songs(sample_df)
sim_df.head(3)

,track_id1,track_id2,sim_score
0,6KRyTDrXw3InXjfnOB3iCR,5Q23x9RyABJ9vx09lNb9hs,0.958976
1,6KRyTDrXw3InXjfnOB3iCR,4zqlCicLICAUkPYYqnPbkn,0.987908
2,6KRyTDrXw3InXjfnOB3iCR,0fonScmHUzC0MglSLn9saA,0.964763


In [12]:
# save these scores to a csv
sim_df.to_csv('sample_song_similarity.csv', index=False)

In [13]:
# save songs in the sample to a csv
sample_df.to_csv('sample_songs.csv', index=False)